<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Proyecto-03---Sistemas-de-Recomendación" data-toc-modified-id="Proyecto-03---Sistemas-de-Recomendación-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Proyecto 03 - Sistemas de Recomendación</a></span><ul class="toc-item"><li><span><a href="#Dataset:-STEAM" data-toc-modified-id="Dataset:-STEAM-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset: STEAM</a></span><ul class="toc-item"><li><span><a href="#Exploración-de-datos" data-toc-modified-id="Exploración-de-datos-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Exploración de datos</a></span></li><li><span><a href="#Filtro-Colaborativo" data-toc-modified-id="Filtro-Colaborativo-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Filtro Colaborativo</a></span></li><li><span><a href="#Para-pensar,-investigar-y,-opcionalmente,-implementar" data-toc-modified-id="Para-pensar,-investigar-y,-opcionalmente,-implementar-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Para pensar, investigar y, opcionalmente, implementar</a></span></li><li><span><a href="#¡Tómate-tiempo-para-investigar-y-leer-mucho!" data-toc-modified-id="¡Tómate-tiempo-para-investigar-y-leer-mucho!-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span><strong>¡Tómate tiempo para investigar y leer mucho!</strong></a></span></li></ul></li></ul></li></ul></div>

# Proyecto 03 - Sistemas de Recomendación

## Dataset: STEAM

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Son dos archivos, uno de calificaciones y otro de información sobre los juegos.**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. Sin embargo, **deberás** modificarlas para hacer un mejor manejo de datos. Algunas cosas a las que deberás prestar atención (tal vez no a todas):
1. Tipos de datos: elige tipos de datos apropiados para cada columna.
2. Descartar columnas poco informativas.
3. Guardar en memoria datasets preprocesados para no tener que repetir código que tarde en correr.

### Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿cuáles son los juegos más populares? ¿Y los menos populares?

### Filtro Colaborativo

Deberás implementar un sistema de recomendación colaborativo para este dataset. Ten en cuenta:

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.

Puedes implementar un filtro colaborativo a partir de la similitud coseno o índice de Jaccard. ¿Puedes utilizar los métodos de la librería Surprise? Si no es así, busca implementaciones (por ejemplo, nuevas librerías) que sean apropiadas.

Para comenzar a trabajar, puedes asumir que cada entrada es un enlace entre una persona usuaria y un item, **independientemente** de si la crítica es buena o mala. 

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Cómo harías para ponerle un valor a la calificación?
1. ¿Cómo harías para agregar contenido? Por ejemplo, cuentas con el género, precio, fecha de lanzamiento y más información de los juegos.
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [2]:
import gc
import gzip
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

def parse(path):
    g = gzip.open(path, "r")
    for l in g:
        yield eval(l)

**Reviews**

In [ ]:
contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n=3, 
# abrira uno de cada tres, y asi.
n = 10
for l in parse(".\Proyectos\Proyecto III\steam_reviews.json.gz"):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1

Cargamos el dataset y elegimos algunas columnas a utilizar.

In [ ]:
#data_reviews = pd.DataFrame(data_reviews, columns=["username", "product_id", "early_access",
#                                                   "hours", "date"])
data_reviews = pd.DataFrame(data_reviews)
reviews = data_reviews.copy()
reviews.head()

In [ ]:
reviews.isna().sum()

Buscamos duplicados

In [ ]:
mask = reviews.duplicated()
print("DUPLICADOS")
reviews[mask]

In [ ]:
name = "Spicy Michael"
reviews.query("username ==  @name")

Eliminamos los duplicados y vemos cantidad de usuarios y de juegos a analizados.

In [ ]:
reviews.drop_duplicates(inplace=True)
print("Total de juegos:", reviews.product_id.value_counts().count())
print("Total de usuarios:", reviews.username.value_counts().count())

In [ ]:
reviews.isna().sum()

Graficamos proporción de juegos con acceso anticipado.

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x=data_reviews["early_access"])
plt.title("ACCESO ANTICIPADO", fontsize=15, fontweight="bold")
plt.tick_params(labelsize=15)
plt.xlabel("Early acces".upper(), labelpad=15)
plt.ylabel("Cantidad".upper(), labelpad=15)
plt.show()

Establecemos algunas consultas para entender un poco más el dataset. agrupamos usuarios y calculamos las horas destinadas a jugar.

In [ ]:
# Cantidad de usuarios sin horas cargadas
horas_null = len(reviews.query("hours.isnull()", engine="python"))
horas_cero = len(reviews.query("hours == 0", engine="python"))
print(f"Hay { horas_cero } de usuarios sin horas cargadas y { horas_null } de usuarios sin registro en horas.")

In [ ]:
reviews.groupby("username").agg(total_juegos=pd.NamedAgg(column="product_id", aggfunc="count"),
                                total_horas=pd.NamedAgg(column="hours", aggfunc="sum"),
                                total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                .sort_values("total_comentarios", ascending=False)

In [ ]:
reviews.groupby("product_id").agg(total_usuarios=pd.NamedAgg(column="username", aggfunc="count"),
                                  total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                  .sort_values("total_usuarios", ascending=False)

In [ ]:
reviews.groupby("product_id").agg(total_usuarios=pd.NamedAgg(column="username", aggfunc="count"),
                                  total_horas=pd.NamedAgg(column="hours", aggfunc="sum"),
                                  total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                  .sort_values("total_usuarios", ascending=False)

**Games**

In [5]:
data_games = []
for games in parse("..\Proyecto III\steam_games.json.gz"):
    data_games.append(games)
#data_games = pd.DataFrame(data_games, columns=[ "id", "app_name", "tags", "specs", "early_access", "sentiment", "metascore", "price" ])

Cargamos el dataset e imprimimos su ```head```.

In [51]:
data_games = pd.DataFrame(data_games)
games = data_games.copy()
games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


Eliminamos las columnas innecesarias y buscamos los valores nulos.

In [52]:
games.drop(columns=["publisher", "genres", "reviews_url", "sentiment", "metascore"], inplace=True)

**Duplicados**

In [53]:
duplicados = len(games[games.drop(columns=["tags", "specs"]).duplicated()])
print(f"Total de registros duplicados: { duplicados }")
id = "612880"
games.query("id == @id")

Total de registros duplicados: 0


,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
13894,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games


In [54]:
games.drop([14573], axis=0, inplace=True)

Es importante remarcar que el ```id``` es la columna que no debemos dejar nula, junto a ```app_name```. Estas son las que nos van a vincular los dos dataset.

In [55]:
games.isna().sum()

app_name              2
title              2050
url                   0
release_date       2067
tags                163
discount_price    31909
specs               670
price              1377
early_access          0
id                    2
developer          3299
dtype: int64

Buscamos los nulos de **id** y **app_name**

In [56]:
games.query('app_name.isna() | id.isna()', engine='python')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
74,NaN,NaN,http://store.steampowered.com/,NaN,NaN,14.99,NaN,19.99,False,NaN,NaN
2580,NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",NaN,"[Single-player, Game demo]",NaN,False,317160,NaN
30961,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Podemos corroborar que el registro 74 comparte nulidad en id y app_name. Prodecemos a eliminarlo y vamos a tratar de completar los datos faltantes de los otros registros. 

In [57]:
games.drop([74], axis=0, inplace=True)
games.query('app_name.isna() | id.isna()', engine='python')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
2580,NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",NaN,"[Single-player, Game demo]",NaN,False,317160,NaN
30961,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


De estos dos registros, el primero de ellos tiene nulo el app_name, en cambio el otro solo el id.

En ambos casos, buscamos a través de la url proporcionada información oficial (web de steam) sobre el nombre del juego o el id. La url de cada juego nos brinda información importante también ya que en la misma incluye el id y el nombre del juego.

In [64]:
print(games.loc[2580,:].url)

http://store.steampowered.com/app/317160/_/


De esta url no podemos inferir el nombre del juego, ingresando en la misma si podemos corroborar esto. El juego en cuestión es un demo llamado **Duet**. Tratamos de inspeccionar en la base de datos información sobre este juego.

In [68]:
app_name = "Duet"
games.query('@app_name == app_name')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
4103,Duet,Duet,http://store.steampowered.com/app/292600/Duet/,2015-08-03,"[Indie, Action, Minimalist, Great Soundtrack, ...",NaN,"[Single-player, Steam Achievements, Full contr...",4.99,False,292600,Kumobius


Antes de eliminarlo deberiamos corroborar que el mismo no tenga ninguna review. De todos modos, es un demo de otro juego por lo tanto podriamos apuntar las reviews al juego final.

In [70]:
print(games.loc[30961,:].url)

http://store.steampowered.com/app/200260


Al inspeccionar la url, verificamos que si corresponde los datos que nos indica el registro.

In [72]:
url = "http://store.steampowered.com/app/200260"
games.query('url == @url')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
30961,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Procedemos a analizar los datos de developer.

In [48]:
developer = 'Rocksteady Studios,Feral Interactive (Mac)'
games.query('developer == @developer')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
1068,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


Al buscar por el developer encontramos un registro correspondiente al mismo juego (30961 y 31617). En este caso,buscamos registros duplicados de los juegos unicamente por medio del nombre del juego.

In [103]:
print(f'Existen { len(games[games["app_name"].duplicated()]) } juegos duplicados por nombre.')
games[games[['app_name', "developer"]].duplicated()].sort_values('app_name', ascending=True)

Existen 38 juegos duplicados por nombre.


,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
30179,Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/219441/Alien...,2013-05-07,[Action],NaN,"[Single-player, Multi-player, Co-op, Downloada...",29.99,False,219441,Gearbox Software
30961,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
13421,Escape Room,NaN,http://store.steampowered.com/app/758210/Escap...,NaN,"[Early Access, Adventure, Simulation, VR]",NaN,"[Single-player, HTC Vive, Tracked Motion Contr...",Free,True,758210,NaN
20989,Invisible Apartment 2,Invisible Apartment 2,http://store.steampowered.com/app/543220/Invis...,2016-10-17,"[Adventure, Indie, Free to Play, Visual Novel,...",NaN,"[Single-player, Downloadable Content]",7.99,False,543220,"Milan Kazarka,Jeroen van Oosten"
31997,MORE SWEATER? OK!,MORE SWEATER? OK!,http://store.steampowered.com/app/746050/MORE_...,2017-12-29,"[Casual, Action, Indie, RPG, Simulation, Singl...",1.59,"[Single-player, Steam Cloud, Stats, Steam Lead...",1.99,False,746050,Triple-Star Studio
1509,The Dream Machine: Chapter 4,The Dream Machine: Chapter 4,http://store.steampowered.com/app/94304/The_Dr...,2013-08-05,"[Adventure, Indie, Casual]",NaN,"[Single-player, Downloadable Content, Steam Tr...",NaN,False,94304,Cockroach Inc.
27089,Total Extreme Wrestling,Total Extreme Wrestling,http://store.steampowered.com/app/344810/Total...,2015-03-19,"[Wrestling, Simulation, Sports, Management]",NaN,[Single-player],19.99,False,344810,Grey Dog Software


Analiza

In [102]:
games.query('app_name == "Aliens: Colonial Marines - Reconnaissance Pack"')

,app_name,title,url,release_date,tags,discount_price,specs,price,early_access,id,developer
30178,Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/224850/Alien...,2013-05-07,[Action],NaN,"[Single-player, Multi-player, Co-op, Downloada...",NaN,False,224850,Gearbox Software
30179,Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/219441/Alien...,2013-05-07,[Action],NaN,"[Single-player, Multi-player, Co-op, Downloada...",29.99,False,219441,Gearbox Software


In [12]:
games.groupby("developer").agg(juegos_publicados=pd.NamedAgg(column="early_access", aggfunc="count")).sort_values("juegos_publicados", ascending=False)

,juegos_publicados
developer,
Ubisoft - San Francisco,1259
"SmiteWorks USA, LLC",813
Dovetail Games,253
"KOEI TECMO GAMES CO., LTD.",232
Paradox Development Studio,156
...,...
Ilja Saburov,1
Ilex Games,1
Ihor Kalinin,1
